In [217]:
from mobilede_utils import request_car_detail_page

html = request_car_detail_page('358672814')

with open('car.html', 'w') as f:
    f.write(html)

In [2]:
from car_mapping import CAR_MAKE_MAP, CAR_MAKE_MODEL_MAP
from mobilede_utils import request_search_page

search_page = request_search_page('volkswagen', 'id.3')

with open('search.html', 'w') as f:
    f.write(search_page)

In [5]:
from mobilede_utils import request_car_detail_page

html = request_car_detail_page('376948323')

In [6]:
from mobilede_parser import extract_car_data

data = extract_car_data(html)
data

{'make': 'BMW',
 'model': 'iX1',
 'color': 'Silver',
 'metallicColor': True,
 'kilometers': '0',
 'price': '74620',
 'registration': None,
 'electric': True,
 'power': '230',
 'batteryCapacity': '00',
 'automatic': True,
 'undamaged': True,
 'seats': '5',
 'leather_upholstery': True,
 'isSUV': True,
 'airconditioning': True,
 'elec_cons': None,
 'n_prev_owners': None,
 'features': ['ABS',
  'Adaptive cornering lights',
  'Adaptive Cruise Control',
  'Alarm system',
  'Alloy wheels',
  'Arm rest',
  'Autom. dimming interior mirror',
  'Blind spot assist',
  'Bluetooth',
  'Central locking',
  'DAB radio',
  'Digital cockpit',
  'Distance warning system',
  'Electric seat adjustment',
  'Electric side mirror',
  'Electric tailgate',
  'Electric windows',
  'Emergency brake assist',
  'Emergency call system',
  'ESP',
  'Fatigue warning system',
  'Fog lamp',
  'Four wheel drive',
  'Hands-free kit',
  'Head-up display',
  'Heated seats',
  'Heated steering wheel',
  'High beam assist',
 

In [ ]:
FEATURES_TO_COLUMN_NAME = {
    'Adaptive Cruise Control': 'parking_sensors',
    'Navigation system': 'satellite_navigator',
    'Adaptive Cruise Control': 'cruise_control_adaptive',
    'Parking sensor: Camera': 'parking_camera_simple_camera',
    'Heated seats': 'seat_heaters',
    'Alloy wheels': 'alloy_wheels',
    'Electric side mirror': 'electric_mirrors',
    'Lane change assist': 'lane_departure_warning_system',
    'Electric tailgate': 'electrically_operated_tailgate',
    'Emergency brake assist': 'emergency_brake_assist',
    'Collision avoidance system': 'collision_avoidance_system',
    'Cruise control: Traditional': 'cruise_control_traditional',
    'Parking sensor: Self-steering systems': 'parking_assistant',
    'Trailer coupling, swiveling': 'tow_bar', # done
    'Trailer coupling, detachable': 'tow_bar', # done
    'Trailer coupling, fix': 'tow_bar', # done
    'Heated steering wheel': 'heated_steering_wheel', # done
    'Sunroof': 'sunroof', # done
    'Panoramic roof': 'sun_hatch_with_panorama', # done
    'Adaptive lighting': 'adaptive_headlights', # done
    'Sport seats': 'sport_seats', # done
    'Auxiliary heating': 'fuel_battery_powered_heater', # done
    'Parking sensor: 360° camera': 'parking_camera_360-degree_camera', # muutetaan kameraksi vain
    'Battery preheating': 'battery_preheating',
    'Electric seat adjustment': 'electric_seats_without_memory', # check
    'Air suspension': 'air_suspension', # done
    'Adaptive cornering lights': 'curve_lights', # done
    'Head-up display': 'head_up_display', # done
    'Sports suspension': 'sport_base', # done,    
}


In [ ]:
result = {
    'make': '',
    'model': '',
    'color': '',
    'driveType': '',
    'price': '',
    'totalOwners': '',
    'kilometers': '',
    'seats': '',
    'power': '',
    'batteryCapacity': '',
    'age': '',
    'isSuv': '',
    'metallicColor': '',
    'airconditioning': '',
    'parking_sensors': '',
    'satellite_navigator': '',
    'cruise_control_adaptive': '',
    'parking_camera_simple_camera': '',
    'seat_heaters': '',
    'driving_assistant': '',
    'alloy_wheels': '',
    'electric_mirrors': '',
    'lane_departure_warning_system': '',
    'electrically_operated_tailgate': '',
    'leather_upholstery': '',
    'emergency_brake_assist': '',
    'collision_avoidance_system': '',
    'cruise_control_traditional': '',
    'parking_assistant': '',
    'tow_bar': '',
    'electric_seats_with_memory': '',
    'heated_steering_wheel': '',
    'sunroof': '',
    'sun_hatch_with_panorama': '',
    'adaptive_headlights': '',
    'sport_seats': '',
    'fuel_battery_powered_heater': '',
    'parking_camera_360-degree_camera': '',
    'battery_preheating': '',
    'electric_seats_without_memory': '',
    'air_suspension': '',
    'curve_lights': '',
    'head_up_display': '',
    'sport_bas': '',
}